In [2]:
import torch as t
print(t.cuda.is_available())

True


In [2]:
import json

def add_svg_tags(data):
    """递归地处理 JSON 数据中的 'response' 字段"""
    if isinstance(data, dict):
        for key, value in data.items():
            if key == 'response':
                # 确保 'response' 字段的值是字符串类型
                if isinstance(value, str):
                    data[key] = f'<svg viewBox="0 0 1024 1024">{value}</svg>'
                else:
                    print(f"Warning: The value of 'response' is not a string: {value}")
            elif isinstance(value, (dict, list)):
                add_svg_tags(value)
    elif isinstance(data, list):
        for item in data:
            add_svg_tags(item)

def modify_json_file(file_path):
    # 读取 JSON 文件
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # 处理 JSON 数据
    add_svg_tags(data)

    # 将修改后的数据写入新文件或覆盖原文件
    new_file_path = file_path.replace('.json', '_modified.json')
    with open(new_file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    
    print(f"Modified JSON saved to {new_file_path}")

modify_json_file("/home/liuzhe/new-files/LoRA-XS/utils/dataset-1024-everypath-9-7.json")

Modified JSON saved to /home/liuzhe/new-files/LoRA-XS/utils/dataset-1024-everypath-9-7_modified.json


In [ ]:
        if labels is not None:
            tokenizer = transformers.AutoTokenizer.from_pretrained(
                '/home/liuzhe/new-files/AI-ModelScope/Mistral-7B-v0___1',
                model_max_length=2048,
                padding_side="right",
                use_fast=True,
            )
                # 创建一个新变量，过滤掉 -100
            filtered_labels = [label for label in labels.tolist()[0] if label != -100]


                # print("input_ids",print(input_ids))
            svg_text = tokenizer.decode(filtered_labels).replace('</s>',"")
            # print("label","svg_text")
            # input_id_decode = tokenizer.decode(f_input_ids).replace('</s>',"")
            # print("Input",input_id_decode)
            png_data = cairosvg.svg2png(bytestring=svg_text)
            # Create an image from the PNG data
            image = Image.open(io.BytesIO(png_data))
            # Create a new image with a light gray background
            background = Image.new('RGB', image.size, (240, 240, 240))  # Light gray color
            background.paste(image, (0, 0), image)
            # Save the final image as JPG
            background.save('/home/liuzhe/new-files/LoRA-XS/utils/output_image1.jpg', 'JPEG')

            # 2. 加载并预处理图像
            image_path = '/home/liuzhe/new-files/LoRA-XS/utils/output_image1.jpg'
            image = Image.open(image_path)

            # 3. 计算图像的嵌入
            inputs = clip_processor(images=image, return_tensors="pt")
            with torch.no_grad():
                image_embeddings = clip_model.get_image_features(**inputs)

            def cosine_similarity(a, b):
                if a.device != b.device:
                    b = b.to(a.device)  # 将 b 移动到 a 的设备上

                a = a / a.norm(dim=-1, keepdim=True)
                a = a.reshape(-1, 1024)
                b = b / b.norm(dim=-1, keepdim=True)
                return (a @ b.T).squeeze().mean()
            similarity = cosine_similarity(hidden_states,image_embeddings)
            print("similarity",similarity)


            # Upcast to float if we need to compute the loss to avoid potential precision issues
            logits = logits.float()
            # print("Logits:", logits)
            # print("Labels:", labels)

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Ensure tensors are on the same device
            shift_labels = shift_labels.to(shift_logits.device)
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits, shift_labels)+(1-similarity)*0.01

In [ ]:
    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        r"""
        Args:
            labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
                Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
                config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
                (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Returns:

        Example:

        ```python
        >>> from transformers import AutoTokenizer, MistralForCausalLM

        >>> model = MistralForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
        >>> tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

        >>> prompt = "Hey, are you conscious? Can you talk to me?"
        >>> inputs = tokenizer(prompt, return_tensors="pt")

        >>> # Generate
        >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
        >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        "Hey, are you conscious? Can you talk to me?\nI'm not conscious, but I can talk to you."
        ```"""

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0]
        print("Shape of hidden_states:", hidden_states.shape)
        logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
        #     tokenizer = transformers.AutoTokenizer.from_pretrained(
        #         '/home/liuzhe/new-files/AI-ModelScope/Mistral-7B-v0___1',
        #         model_max_length=2048,
        #         padding_side="right",
        #         use_fast=True,
        #     )
        # #     # 创建一个新变量，过滤掉 -100
        #     filtered_labels = [label for label in labels.tolist()[0] if label != -100]
        # #     # print("input_ids",print(input_ids))
        #     svg_text = tokenizer.decode(filtered_labels).replace('</s>',"")
        #     png_data = cairosvg.svg2png(bytestring=svg_text)
        #     # Create an image from the PNG data
        #     image = Image.open(io.BytesIO(png_data))
        #     # Create a new image with a light gray background
        #     background = Image.new('RGB', image.size, (240, 240, 240))  # Light gray color
        #     background.paste(image, (0, 0), image)
        #     # Save the final image as JPG
        #     background.save('/home/liuzhe/new-files/LoRA-XS/utils/output_image1.jpg', 'JPEG')



            # Shift so that tokens < n predict n            # Shift so that tokens < n predict n
            # filtered_labels = torch.tensor(filtered_labels, device=self.device)
            # labels_embeds = self.model.embed_tokens(filtered_labels)
            # # 打印labels的embedding shape，用于调试
            # print("Labels embedding shape:", labels_embeds.shape)
            shift_logits = logits[..., :-1, :].contiguous()
            # print("forward:labels size:", labels.size())  
            # print("forward:labels content:", labels)    
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Ensure tensors are on the same device
            shift_labels = shift_labels.to(shift_logits.device)
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits, shift_labels)